# An SQL notebook
### to note SQL basic techniques

Contents:
  - SQL and database basics

### Broad import statement

In [2]:
# data sourcing:
import csv

# databases:
import sqlite3

# data toolkit:
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt

# CHEATSHEETS
[SQL Cheatsheet](https://learnsql.com/blog/ultimate-sql-cheat-sheet/All-sql-cheat-sheet-a4.pdf)

[Numpy Cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf)

[Pandas Cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)

#

# SQL and database basics
## use db:
### https://www.kaggle.com/datasets/hugomathien/soccer
 

### [SQL Cheatsheet](https://learnsql.com/blog/ultimate-sql-cheat-sheet/All-sql-cheat-sheet-a4.pdf)

### DBeaver:

In [9]:
# install dbeaver with
# sudo add-apt-repository ppa:serge-rider/dbeaver-ce
# sudo apt-get update
# sudo apt-get install dbeaver-ce

# run from the command line with:
# dbeaver-ce

In [10]:
# order of SQL operations matters:

![image.png](https://cdn.sisense.com/wp-content/uploads/image-1-order-blog.png)

In [11]:
# python access to database:
db_path_movies = '../data/movies.sqlite'
# import sqlite3
conn = sqlite3.connect(db_path_movies)
c = conn.cursor()

In [12]:
# SQL run the commands:
c.execute("SELECT * FROM directors")
rows = c.fetchall()
first_row = rows[0]
first_row[0]

'Georges Méliès'

In [13]:
# SQL fetch list of row elements:
conn = sqlite3.connect(db_path_movies)
conn.row_factory = sqlite3.Row
c = conn.cursor()

c.execute("SELECT * FROM directors")
rows = c.fetchall()
first_row = rows[0]

first_row['birth_year'] # table field names: name, birth_year, death_year, imdb_director_id, id
first_row.keys()

['name', 'birth_year', 'death_year', 'imdb_director_id', 'id']

In [14]:
tuple(first_row)

('Georges Méliès', 1861, 1938, 'nm0617588', 1)

In [15]:
list(first_row)

['Georges Méliès', 1861, 1938, 'nm0617588', 1]

In [16]:
# SQL use fetchone when relevant:
c.execute("SELECT * FROM directors WHERE directors.imdb_director_id = 'nm0000186'")
row = c.fetchone()
print(row[0], '-' ,row[1])

David Lynch - 1946


In [17]:
# SQL Projection - select columns in the query:
query = "SELECT d.id, d.name, d.birth_year FROM directors AS d"
# NB - alias

# SQL Selection- select row:
query = """
SELECT *
FROM directors AS d
WHERE d.name = 'D.W. Griffith'
"""
c.execute(query)
row = c.fetchone()
print(f'{row[0]} {row[1]} {row[2]} {row[3]} {row[4]}')

D.W. Griffith 1875 1948 nm0000428 4


In [18]:
# SQL  multiple selections:

query ="""SELECT *
FROM directors AS d
WHERE d.name = 'D.W. Griffith'
OR d.name = 'David Lynch'"""

query = """SELECT *
FROM directors AS d
WHERE d.name IN ('D.W. Griffith', 'David Lynch')"""

query ="""SELECT *
FROM movies AS m
WHERE UPPER(m.title) LIKE 'THE %'""" # all movies starting with "THE "

In [19]:
# SQL counting
q = """SELECT COUNT(directors.id)
FROM directors
WHERE directors.birth_year >= 1980
"""

In [20]:
# SQL sorting:
q = """SELECT *
FROM directors AS d
ORDER BY d.birth_year DESC
LIMIT 10"""

In [ ]:
# SQL select unique values with DISTINCT:
q = """SELECT DISTINCT *
FROM directors AS d
ORDER BY d.birth_year DESC
LIMIT 10"""

In [ ]:
# combine lists with SQL UNION:
q = """SELECT DISTINCT *
FROM directors
UNION
SELECT * FROM movies
"""

In [ ]:
# simple aggregation with MAX"
q = """
SELECT MAX(birth_year) FROM directors
"""

In [ ]:
# subquery to select single row data with criteria
q = """
SELECT name, age FROM directors
WHERE age IN (SELECT MAX(age) FROM directors)
"""

In [21]:
query = """SELECT d.name
            FROM directors AS d
            ORDER BY d.name
            LIMIT 10"""
c.execute(query)
rows = c.fetchall()
first_row = rows[0]

print(rows)

# for r in rows:
#     print(f"{r[0]}")

[<sqlite3.Row object at 0x7f5326fdaa10>, <sqlite3.Row object at 0x7f53082636a0>, <sqlite3.Row object at 0x7f5308263100>, <sqlite3.Row object at 0x7f5308263a30>, <sqlite3.Row object at 0x7f5308263f10>, <sqlite3.Row object at 0x7f5308263d00>, <sqlite3.Row object at 0x7f5308262e90>, <sqlite3.Row object at 0x7f53082631c0>, <sqlite3.Row object at 0x7f5308263a00>, <sqlite3.Row object at 0x7f5308262e00>]


In [22]:
db_path_soccer = '../data/soccer.sqlite'
conn = sqlite3.connect(db_path_soccer)
conn.row_factory = sqlite3.Row
c = conn.cursor()

In [23]:
# GROUPING:
# Grouping rows on a given column C (aggregating rows with a function where values of C column are the same)

# how many matches played per country:
query = """SELECT COUNT(matches.id), matches.country_id
FROM "Match" AS matches
GROUP BY matches.country_id"""

c.execute(query)
row = c.fetchone()
print(f'{row[0]}')

1728


In [24]:
# What if we want to sort those results? We need an alias:
query1 = """SELECT COUNT(matches.id) AS match_count, matches.country_id
FROM "Match" AS matches
GROUP BY matches.country_id
ORDER BY match_count DESC"""

query2 = """SELECT COUNT(matches.id) AS match_count, matches.country_id
FROM "Match" AS matches
GROUP BY matches.country_id
HAVING match_count >= 3000
ORDER BY match_count DESC"""

c.execute(query2)
rows = c.fetchall()

for r in rows:
    print(f"Matches: {r['match_count']} Country id: {r['country_id']}")


Matches: 3040 Country id: 21518
Matches: 3040 Country id: 4769
Matches: 3040 Country id: 1729
Matches: 3017 Country id: 10257


In [25]:
# naming outcomes and calculating using CASE & END AS

# note conditional equals in SQL as single =

# How many matches were
    # won by the home team?
    # won by the away team?
    # finished with a draw?

query = """
SELECT
COUNT(matches.id) AS outcome_count,
CASE
    WHEN matches.home_team_goal > matches.away_team_goal
        THEN 'home_win'
    WHEN matches.home_team_goal = matches.away_team_goal
        THEN 'draw'
    ELSE 'away_win'
END AS outcome
FROM "Match" AS matches
GROUP BY outcome
ORDER BY outcome_count DESC"""

c.execute(query)
rows = c.fetchall()
first_row = rows[0]
first_row.keys()

for r in rows:
    print(f"{r['outcome_count']} {r['outcome']}")


11917 home_win
7466 away_win
6596 draw


In [26]:
# Querying multiple tables:

query = """
SELECT League.name as LeagueName, Country.name as CountryName
FROM League
JOIN Country ON League.country_id = Country.id"""

c.execute(query)
rows = c.fetchall()
first_row = rows[0]
first_row.keys()

for r in rows:
    print(f" {r['LeagueName']}") # including CountryName looks like a duplicate

 Belgium Jupiler League
 England Premier League
 France Ligue 1
 Germany 1. Bundesliga
 Italy Serie A
 Netherlands Eredivisie
 Poland Ekstraklasa
 Portugal Liga ZON Sagres
 Scotland Premier League
 Spain LIGA BBVA
 Switzerland Super League


In [27]:
# string selection with SQL with (movies db):
exclude = 'cloverfield'
query = f'''SELECT m.title
        FROM movies AS m
        WHERE UPPER(title) LIKE '% LOVE %'
        OR UPPER(title) LIKE 'LOVE %'
        OR UPPER(title) LIKE 'LOVE,%'
        AND m.title NOT LIKE "%{exclude}%"
        ORDER BY m.title'''

In [28]:
# doing maths on the fly and naming columns:
query = """SELECT d.name,
            (m.start_year - d.birth_year) age
            FROM directors AS d
            JOIN movies AS m ON m.director_id = d.id
            WHERE age IS NOT NULL
            ORDER BY age
            LIMIT 5
            """

In [29]:
# numerical bucketing is simply done this way:
# 30 buckets achieved by / 30 + 1, with *30 gives the labels to time_range
query = """SELECT
        (minutes / 30 + 1)*30 time_range,
        COUNT(*)
    FROM movies
    WHERE minutes IS NOT NULL
    GROUP BY time_range"""
    # neat and quick

In [30]:
# both the following queries achieve the same outcome
# 'query' was my solution, I just needed the LEFT JOIN to achieve the same as
# 'q_sol'
# the WITH new_table_name AS (SELECT * FROM table_x etc) is powerful
# this allows additional table definition on the fly

# The SQL Command: COALESCE returns the first non-null result in the list
# Used here to ensure 0 is returned if the count is null

query = f"""SELECT
        c.ContactName AS contact_name,
        COUNT(o.OrderID) NoOrders
        FROM Customers AS c
        LEFT JOIN Orders as o ON o.CustomerID = c.CustomerID
        GROUP BY contact_name
        ORDER BY NoOrders
        """

q_sol = """
    WITH no_orders AS (
        SELECT *
        FROM Customers c
        LEFT JOIN Orders o ON o.CustomerID = c.CustomerID
    )
    SELECT
        no_orders.ContactName
        ,COALESCE(COUNT(no_orders.OrderID), 0) AS number_orders
    FROM no_orders
    GROUP BY no_orders.CustomerID
    ORDER BY number_orders
    """

In [31]:
# Window Functions like RANK or SUM with the OVER keyword.
# https://mode.com/sql-tutorial/sql-window-functions

'''Implement order_rank_per_customer to rank the orders of each
customer according to the order date.
For each customer, the orders should be ranked in the chronological order.
This function should return a list of tuples like
(OrderID, CustomerID, OrderDate, OrderRank).'''

# Simple rank in order of something, in this case date

query = f"""SELECT
    o.OrderID,
    c.CustomerID,
    o.OrderDate,
    RANK() OVER (PARTITION BY c.CustomerID
        ORDER BY o.OrderDate)
        AS OrderRank
    FROM Orders as o
    JOIN Customers AS c ON c.CustomerID = o.CustomerID
    """

'''Implement order_cumulative_amount_per_customer to compute the
cumulative amount (in USD) of the orders of each customer according
to the order date.
For each customer, the orders should be ranked in the chronological order.
This function should return a list of tuples like
(OrderID, CustomerID, OrderDate, OrderCumulativeAmount).'''

#

query = f"""
    WITH order_sum AS (
        SELECT
        o.OrderID,
        o.CustomerID,
        o.OrderDate,
        SUM(ROUND(od.UnitPrice * od.Quantity,2))
            OVER (PARTITION BY o.CustomerID
            ORDER BY o.OrderDate)
            AS OrderCumulativeAmount
        FROM Orders as o
        JOIN OrderDetails AS od ON od.OrderID = o.OrderID
        )
    SELECT *
    FROM order_sum
    GROUP BY order_sum.OrderID
    ORDER BY order_sum.CustomerID
    """

In [32]:
# this shows double WITH, producing 2 tables, the general average just being a
# number

# TASK:
# return the customers who have an average purchase greater
# than the general average purchase
query = '''
    WITH OrderValues AS (
        SELECT
        SUM(od.UnitPrice * od.Quantity) AS value,
        od.OrderID
        FROM OrderDetails od
        GROUP BY od.OrderID
    ),
    generalaverage AS (
        SELECT
            ROUND(AVG(ov.value), 0) AS average
        FROM OrderValues ov
        )
    SELECT
        c.CustomerID
        ,ROUND(AVG(ov.value),2) AS customer_average
    FROM Customers as c
    JOIN Orders AS o ON o.CustomerID = c.CustomerID
    JOIN ordervalues AS ov ON ov.OrderID = o.OrderID
    GROUP BY c.CustomerID
    HAVING customer_average > (SELECT average FROM generalaverage)
    ORDER BY customer_average DESC
    '''

In [ ]:
# RANDOM CUT AND PASTE FROM
# https://pgexercises.com/gettingstarted.html

In [ ]:
# simple join 1

postgreSQL_QUERY = '''
SELECT b.starttime AS start, m.surname AS name
FROM cd.members AS m
	INNER JOIN cd.bookings AS b
	ON m.memid = b.memid
WHERE
m.surname IN ('Farrell')
 AND m.firstname IN ('David')
ORDER BY b.starttime;'''

# simple join 2

postgreSQL_QUERY = '''
SELECT b.starttime AS start, f.name AS name
FROM cd.facilities AS f
	INNER JOIN cd.bookings AS b
	ON f.facid = b.facid
WHERE
f.name IN ('Tennis Court 2','Tennis Court 1') AND
b.starttime >= '2012-09-21' AND b.starttime < '2012-09-22'
ORDER BY b.starttime;
'''

# simple join 3 - self
# How can you output a list of all members who have recommended another member?
# Ensure that there are no duplicates in the list,
# and that results are ordered by (surname, firstname).
postgreSQL_QUERY = '''
SELECT DISTINCT m.firstname, m.surname
FROM cd.members AS m
	INNER JOIN cd.members AS m2
	ON m.memid = m2.recommendedby
ORDER BY m.surname;'''

# simple join 4 - self2
# How can you output a list of all members who have recommended another member?
# Ensure that there are no duplicates in the list, and that results
# are ordered by (surname, firstname).
postgreSQL_QUERY = '''
SELECT
m.firstname AS memfname
, m.surname AS memsname
, m2.firstname AS recfname
, m2.surname AS recsname
FROM cd.members AS m
	LEFT OUTER JOIN cd.members AS m2
	ON m2.memid = m.recommendedby
ORDER BY memsname, memfname;
'''

#  Produce a list of all members who have used a tennis court
# Simple Join 4 - threejoin
# How can you produce a list of all members who have used a tennis court?
# Include in your output the name of the court, and the name of the member formatted as a single column.
# Ensure no duplicate data, and order by the member name followed by the facility name.
build_up = '''
SELECT f.name AS facility
FROM cd.facilities AS f
WHERE f.name LIKE '%Tennis Court%'

SELECT CONCAT_WS(' ', m.firstname, m.surname) AS member
FROM cd.members AS m
'''

postgreSQL_QUERY = '''
SELECT DISTINCT CONCAT_WS(' ', m.firstname, m.surname) AS member, f.name AS facility
FROM cd.members AS m
JOIN cd.bookings AS b
ON m.memid = b.memid
JOIN cd.facilities AS f
ON f.facid = b.facid
WHERE f.name LIKE '%Tennis Court%'
ORDER BY member, facility
'''

# alternatively:
# NB  || operator is used to concatenate strings.
# with a string in-between: e.g: mems.firstname || ' ' || mems.surname
'''
select distinct mems.firstname || ' ' || mems.surname as member, facs.name as facility
	from
		cd.members mems
		inner join cd.bookings bks
			on mems.memid = bks.memid
		inner join cd.facilities facs
			on bks.facid = facs.facid
	where
		facs.name in ('Tennis Court 2','Tennis Court 1')
order by member, facility
'''

# Produce a list of costly bookings
# Threejoin2
# How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30?
# Remember that guests have different costs to members (the listed costs are per half-hour 'slot'),
# and the guest user is always ID 0.
# Include in your output the name of the facility, the name of the
# member formatted as a single column, and the cost.
# Order by descending cost, and do not use any subqueries.

postgreSQL_QUERY = '''

'''